In [1]:
# filename: output.ipynb
# purpose: generate output data

# OHT noise, normal and outlier data generation

### Outlier detection method
- Outlier data will be detected and generated based on Moving Average and Moving Standard Deviation 

### Processing flow
- Read Raw table, ohtraw from duckdb filedb, which was created in previous step, parse notebook
- Create a Work table, work in memory duckdb with additional columns MVAVG_,MVSTD_,MVSIG_,FLAG columns.
- Calculate Moving Average, Standard Devidation
- Calculate Sigma value of the column data based on Moving Average and Standard Deviation
- Calcuate FLAG column value based on Sigma value
- Fetch the work table into work dataframe
- Split noise and normal dataframe from work dataframe based on FLAG value
- Create outlier dataframe based on normal dataframe
- Update outlier dataframe by applying outlier pattern
- Calculate Moving Average, Standard Deviation, Sigma value for outlier data
- Prepare mix dataset with normal and outlier for ML 
- Save noise, normal, outler, mix dataframe to duckdb tables for later graphing.
- Save noise, normal, outler, mix csvfle
- Check csvfile size 

In [2]:
# packages
import time
import pathlib
import textwrap
import pandas as pd

import humanfriendly as human
import duckdb

import ohtconf as conf
import ohtcomm as comm

## Main

In [3]:
mainstart = time.time()

### Prepare in-memory work table base on in-file raw table 

In [4]:
# open in-memory db

con = duckdb.connect(database=":memory:")

In [5]:
# create in-memory work table

query = "DROP TABLE IF EXISTS work"
con.execute(query)

coldef = ""
for name, dtype in zip(conf.COLUMN_NAMES, conf.COLUMN_DBTYPES):
    if coldef:
        coldef += ", " + name + " " + dtype
    else:
        coldef += name + " " + dtype

for col in conf.COLUMN_GRAPH:
    coldef += ", " + conf.MVAVG + col + " " + "FLOAT"
    coldef += ", " + conf.MVSTD + col + " " + "FLOAT"
    coldef += ", " + conf.MVSIG + col + " " + "INTEGER"

coldef += ", " + f"{conf.COLUMN_FLAG}  INTEGER"

query = f"CREATE TABLE work ( {coldef} )"
# print(textwrap.fill(query, width=120))

con.execute(query)

In [6]:
# attach in-file db, raw table was prepared in the previous step, parse

con.execute(f"ATTACH DATABASE '{conf.DBFILE}' AS filedb (READ_ONLY)")

In [7]:
# insert into in-memory work table from in-file raw table with additional Moving Avg,Std and Flag=0

coldef = ", ".join(conf.COLUMN_NAMES)

for col in conf.COLUMN_GRAPH:
    coldef += (
        ", "
        + f"AVG({col}) OVER (ORDER BY {conf.COLUMN_NAMES[0]} ROWS BETWEEN {conf.POINTS['MOVING']} PRECEDING AND CURRENT ROW) AS {conf.MVAVG}{col}"
    )
    coldef += (
        ", "
        + f"STDDEV({col}) OVER (ORDER BY {conf.COLUMN_NAMES[0]} ROWS BETWEEN {conf.POINTS['MOVING']} PRECEDING AND CURRENT ROW) AS {conf.MVSTD}{col}"
    )
    coldef += ", 0"  # mvsig_
coldef += ", 0"  # flag

query = f"INSERT INTO work SELECT {coldef} FROM filedb.{conf.TABNAME_RAW} ORDER BY {conf.COLUMN_NAMES[0]}"
# print(textwrap.fill(query, width=120))

con.execute(query)

In [8]:
# detach filedb

con.execute("DETACH DATABASE filedb")

In [9]:
# Update NULL value after window function
query = f"SELECT * FROM work ORDER BY {conf.COLUMN_NAMES[0]}"
dfwork = con.execute(query).df()
dfwork.bfill(inplace=True)

# recreate work table base one work dataframe
con.execute("DROP TABLE IF EXISTS work")
con.execute("CREATE TABLE work AS SELECT * FROM dfwork")

In [10]:
# calculate Sigma value

coldef = ""
for col in conf.COLUMN_GRAPH:
    setdef = textwrap.dedent(f"""{conf.MVSIG}{col} = CASE 
                  WHEN {col} >= ({col} - 1 * {conf.MVSTD}{col}) AND {col} <= ({col} + 1 * {conf.MVSTD}{col})  then 1
                  WHEN {col} >= ({col} - 2 * {conf.MVSTD}{col}) AND {col} <= ({col} + 2 * {conf.MVSTD}{col})  then 2
                  WHEN {col} >= ({col} - 3 * {conf.MVSTD}{col}) AND {col} <= ({col} + 3 * {conf.MVSTD}{col})  then 3
                  WHEN {col} >= ({col} - 4 * {conf.MVSTD}{col}) AND {col} <= ({col} + 4 * {conf.MVSTD}{col})  then 4
                  WHEN {col} >= ({col} - 5 * {conf.MVSTD}{col}) AND {col} <= ({col} + 5 * {conf.MVSTD}{col})  then 5
                  ELSE 6
                  END""")
    if not coldef:
        coldef = setdef
    else:
        coldef += f", {setdef}"

query = f"UPDATE work SET {coldef}"
# print(textwrap.fill(query, width=120))

con.execute(query)

In [11]:
# calculate Flag=1 based on configured sigma value

coldef = ""
for col in conf.COLUMN_GRAPH:
    if not coldef:
        coldef = f"{conf.MVSIG}{col} >= {conf.SIGMA_NOISE}"
    else:
        coldef += f" OR {conf.MVSIG}{col} >= {conf.SIGMA_NOISE}"

query = f"UPDATE work SET {conf.COLUMN_FLAG}=1 WHERE {coldef}"
# print(textwrap.fill(query, width=120))

con.execute(query)

### Prepare noise, normal dataframe

In [12]:
# fetch table from work table based on Flag

dfnoise = con.execute(f"SELECT * FROM work WHERE {conf.COLUMN_FLAG}!=0 ORDER BY {conf.COLUMN_NAMES[0]}").df()
dfnorm = con.execute(f"SELECT * FROM work WHERE {conf.COLUMN_FLAG}=0 ORDER BY {conf.COLUMN_NAMES[0]}").df()

# round float type value
dfnoise = dfnoise.round(1)
dfnorm = dfnorm.round(1)

print(f"row count, noise={len(dfnoise)}, normal={len(dfnorm)}, noise ratio={len(dfnoise)/(len(dfnoise)+len(dfnorm))}")

row count, noise=0, normal=108000, noise ratio=0.0


In [13]:
# set float display format
pd.set_option("display.float_format", "{:.1f}".format)

In [14]:
dfnoise.head()

,DATETM,TEM,PM1,PM2_5,PM10,CO,NH3,CT1,CT2,CT3,CT4,MVAVG_TEM,MVSTD_TEM,MVSIG_TEM,MVAVG_PM1,MVSTD_PM1,MVSIG_PM1,MVAVG_PM2_5,MVSTD_PM2_5,MVSIG_PM2_5,MVAVG_PM10,MVSTD_PM10,MVSIG_PM10,MVAVG_CO,MVSTD_CO,MVSIG_CO,MVAVG_NH3,MVSTD_NH3,MVSIG_NH3,MVAVG_CT1,MVSTD_CT1,MVSIG_CT1,MVAVG_CT2,MVSTD_CT2,MVSIG_CT2,MVAVG_CT3,MVSTD_CT3,MVSIG_CT3,MVAVG_CT4,MVSTD_CT4,MVSIG_CT4,FLAG


In [15]:
dfnorm.head()

,DATETM,TEM,PM1,PM2_5,PM10,CO,NH3,CT1,CT2,CT3,CT4,MVAVG_TEM,MVSTD_TEM,MVSIG_TEM,MVAVG_PM1,MVSTD_PM1,MVSIG_PM1,MVAVG_PM2_5,MVSTD_PM2_5,MVSIG_PM2_5,MVAVG_PM10,MVSTD_PM10,MVSIG_PM10,MVAVG_CO,MVSTD_CO,MVSIG_CO,MVAVG_NH3,MVSTD_NH3,MVSIG_NH3,MVAVG_CT1,MVSTD_CT1,MVSIG_CT1,MVAVG_CT2,MVSTD_CT2,MVSIG_CT2,MVAVG_CT3,MVSTD_CT3,MVSIG_CT3,MVAVG_CT4,MVSTD_CT4,MVSIG_CT4,FLAG
0,2024-07-29 09:43:18.696,40.8,10,12,13,161,88,0.8,1.0,0.5,0.6,40.8,0.0,1,10.0,0.0,1,12.0,0.0,1,13.0,0.0,1,161.0,0.0,1,88.0,0.0,1,0.8,0.0,1,1.0,0.0,1,0.5,0.0,1,0.6,0.0,1,0
1,2024-07-29 09:43:18.781,40.8,10,12,13,161,88,0.8,1.0,0.5,0.6,40.8,0.0,1,10.0,0.0,1,12.0,0.0,1,13.0,0.0,1,161.0,0.0,1,88.0,0.0,1,0.8,0.0,1,1.0,0.0,1,0.5,0.0,1,0.6,0.0,1,0
2,2024-07-29 09:43:18.883,40.8,10,12,13,161,88,0.8,1.0,0.5,0.5,40.8,0.0,1,10.0,0.0,1,12.0,0.0,1,13.0,0.0,1,161.0,0.0,1,88.0,0.0,1,0.8,0.0,1,1.0,0.0,1,0.5,0.0,1,0.6,0.1,1,0
3,2024-07-29 09:43:18.989,40.8,10,12,13,161,88,0.9,1.0,0.5,0.6,40.8,0.0,1,10.0,0.0,1,12.0,0.0,1,13.0,0.0,1,161.0,0.0,1,88.0,0.0,1,0.8,0.0,1,1.0,0.0,1,0.5,0.0,1,0.6,0.1,1,0
4,2024-07-29 09:43:19.094,40.8,10,12,13,161,88,0.8,1.0,0.7,0.6,40.8,0.0,1,10.0,0.0,1,12.0,0.0,1,13.0,0.0,1,161.0,0.0,1,88.0,0.0,1,0.8,0.0,1,1.0,0.0,1,0.5,0.1,1,0.6,0.0,1,0


### Prepare outlier dataframe from normal dataframe

In [16]:
# choose dfoutl data in dfnorm

# reset datetm value: drop any duplicates and missing.
# keep datetm uniqueness in between normal and outlier to merge later to keep moving avg
dfnorm[conf.COLUMN_NAMES[0]] = pd.date_range(start="2024-01-01", periods=len(dfnorm), freq="100ms")

# take outlier candidates in splitted in datetm as moving avg is different in datetm range.
alls = [dfnorm.iloc[i : i + conf.POINTS["PATTERN"]] for i in range(0, len(dfnorm), conf.POINTS["PATTERN"])]
outls = alls[::4]

# index after split by datetm ordering
dfoutl = pd.concat(outls)
dfoutl = dfoutl.sort_values(by=conf.COLUMN_NAMES[0])
dfoutl = dfoutl.reset_index(drop=True)

# filter dfnorm to keep only rows where datetm is not in dfoutl
dfnorm = dfnorm[~dfnorm[conf.COLUMN_NAMES[0]].isin(dfoutl[conf.COLUMN_NAMES[0]])]

# index after split by datetm ordering
dfnorm = dfnorm.sort_values(by=conf.COLUMN_NAMES[0])
dfnorm = dfnorm.reset_index(drop=True)

print(f"row count, outlier={len(dfoutl)}, normal={len(dfnorm)}, outlier ratio={len(dfoutl)/(len(dfoutl)+len(dfnorm))}")

row count, outlier=27000, normal=81000, outlier ratio=0.25


In [17]:
# Update outlier
_start = time.time()

dfoutl = comm.gen_outlier(dfoutl)

# round float type value
dfoutl = dfoutl.round(1)

_elapsed = time.time() - _start
print(f"get_outlier elapsed time: {human.format_timespan(_elapsed)}")
# 14 min 16 sec, 3_280_186 rows, INPUT_MAXSIZE=650MB, exclude DATETM on csvfile

updated outlier count=1 / 27000
get_outlier elapsed time: 7.59 seconds


In [18]:
# recreate work table base one dfoutl
con.execute("DROP TABLE IF EXISTS work")
con.execute("CREATE TABLE work AS SELECT * FROM dfoutl WHERE 1=0")

In [19]:
# insert into in-memory work table from dfoutl with calculated Moving Avg,Std and asis Flag

coldef = ", ".join(conf.COLUMN_NAMES)

for col in conf.COLUMN_GRAPH:
    coldef += (
        ", "
        + f"AVG({col}) OVER (ORDER BY {conf.COLUMN_NAMES[0]} ROWS BETWEEN {conf.POINTS['MOVING']} PRECEDING AND CURRENT ROW) AS {conf.MVAVG}{col}"
    )
    coldef += (
        ", "
        + f"STDDEV({col}) OVER (ORDER BY {conf.COLUMN_NAMES[0]} ROWS BETWEEN {conf.POINTS['MOVING']} PRECEDING AND CURRENT ROW) AS {conf.MVSTD}{col}"
    )
    coldef += ", 0"  # mvsig_
coldef += ", FLAG"  # flag

query = f"INSERT INTO work SELECT {coldef} FROM dfoutl ORDER BY {conf.COLUMN_NAMES[0]}"
# print(textwrap.fill(query, width=120))

con.execute(query)

In [20]:
# Update NULL value after window function
query = f"SELECT * FROM work ORDER BY {conf.COLUMN_NAMES[0]}"
dfoutl = con.execute(query).df()
dfoutl.bfill(inplace=True)

# recreate work table base one work dataframe
con.execute("DROP TABLE IF EXISTS work")
con.execute("CREATE TABLE work AS SELECT * FROM dfoutl")

In [21]:
# calculate Sigma value

coldef = ""
for col in conf.COLUMN_GRAPH:
    setdef = textwrap.dedent(f"""{conf.MVSIG}{col} = CASE 
                  WHEN {col} >= ({col} - 1 * {conf.MVSTD}{col}) AND {col} <= ({col} + 1 * {conf.MVSTD}{col})  then 1
                  WHEN {col} >= ({col} - 2 * {conf.MVSTD}{col}) AND {col} <= ({col} + 2 * {conf.MVSTD}{col})  then 2
                  WHEN {col} >= ({col} - 3 * {conf.MVSTD}{col}) AND {col} <= ({col} + 3 * {conf.MVSTD}{col})  then 3
                  WHEN {col} >= ({col} - 4 * {conf.MVSTD}{col}) AND {col} <= ({col} + 4 * {conf.MVSTD}{col})  then 4
                  WHEN {col} >= ({col} - 5 * {conf.MVSTD}{col}) AND {col} <= ({col} + 5 * {conf.MVSTD}{col})  then 5
                  ELSE 6
                  END""")
    if not coldef:
        coldef = setdef
    else:
        coldef += f", {setdef}"

query = f"UPDATE work SET {coldef}"
# print(textwrap.fill(query, width=120))

con.execute(query)

In [22]:
dfoutl = con.execute(f"SELECT * FROM work ORDER BY {conf.COLUMN_NAMES[0]}").df()

In [23]:
# outlier first pattern range data
dfoutl.iloc[conf.POINTS["MOVING"] : conf.POINTS["MOVING"] + conf.POINTS["PATTERN"]]

,DATETM,TEM,PM1,PM2_5,PM10,CO,NH3,CT1,CT2,CT3,CT4,MVAVG_TEM,MVSTD_TEM,MVSIG_TEM,MVAVG_PM1,MVSTD_PM1,MVSIG_PM1,MVAVG_PM2_5,MVSTD_PM2_5,MVSIG_PM2_5,MVAVG_PM10,MVSTD_PM10,MVSIG_PM10,MVAVG_CO,MVSTD_CO,MVSIG_CO,MVAVG_NH3,MVSTD_NH3,MVSIG_NH3,MVAVG_CT1,MVSTD_CT1,MVSIG_CT1,MVAVG_CT2,MVSTD_CT2,MVSIG_CT2,MVAVG_CT3,MVSTD_CT3,MVSIG_CT3,MVAVG_CT4,MVSTD_CT4,MVSIG_CT4,FLAG
600,2024-01-01 00:04:00.000,40.6,10,13,14,163,89,0.9,1.0,0.4,0.6,43.9,9.9,1,16.2,12.5,1,14.7,8.4,1,14.2,2.3,1,169.7,35.1,1,100.3,41.8,1,1.0,0.7,1,1.3,1.1,1,0.8,1.1,1,0.6,0.3,1,3
601,2024-01-01 00:04:00.100,40.5,12,13,13,164,89,0.8,1.0,0.5,0.5,43.9,9.9,1,16.2,12.5,1,14.7,8.4,1,14.2,2.3,1,169.7,35.1,1,100.3,41.8,1,1.0,0.7,1,1.3,1.1,1,0.8,1.1,1,0.6,0.3,1,3
602,2024-01-01 00:04:00.200,40.5,12,14,13,162,88,0.8,1.0,0.5,0.6,43.9,9.9,1,16.3,12.5,1,14.7,8.4,1,14.2,2.3,1,169.7,35.1,1,100.3,41.8,1,1.0,0.7,1,1.3,1.1,1,0.8,1.1,1,0.6,0.3,1,3
603,2024-01-01 00:04:00.300,40.5,10,15,14,162,88,0.9,1.0,0.4,0.5,43.9,9.9,1,16.2,12.5,1,14.7,8.4,1,14.2,2.3,1,169.7,35.1,1,100.3,41.8,1,1.0,0.7,1,1.3,1.1,1,0.8,1.1,1,0.6,0.3,1,3
604,2024-01-01 00:04:00.400,40.6,11,15,14,164,88,0.9,1.0,0.4,0.6,43.9,9.9,1,16.2,12.5,1,14.7,8.4,1,14.2,2.3,1,169.7,35.1,1,100.3,41.8,1,1.0,0.7,1,1.3,1.1,1,0.8,1.1,1,0.6,0.3,1,3
605,2024-01-01 00:04:00.500,40.5,11,16,14,162,89,0.8,1.0,0.4,0.6,43.9,9.9,1,16.2,12.5,1,14.7,8.4,1,14.2,2.3,1,169.7,35.1,1,100.3,41.8,1,1.0,0.7,1,1.3,1.1,1,0.8,1.1,1,0.6,0.3,1,3
606,2024-01-01 00:04:00.600,40.6,11,18,15,162,89,0.8,1.0,0.5,0.6,43.9,9.9,1,16.2,12.5,1,14.7,8.4,1,14.2,2.3,1,169.7,35.1,1,100.3,41.8,1,1.0,0.7,1,1.3,1.1,1,0.8,1.1,1,0.6,0.3,1,3
607,2024-01-01 00:04:00.700,40.5,12,19,15,163,88,0.9,1.0,0.4,0.5,43.9,9.9,1,16.2,12.5,1,14.7,8.4,1,14.2,2.3,1,169.7,35.1,1,100.3,41.8,1,1.0,0.7,1,1.3,1.1,1,0.8,1.1,1,0.6,0.3,1,3
608,2024-01-01 00:04:00.800,40.5,10,19,14,161,89,0.9,1.0,0.4,0.5,43.9,9.9,1,16.2,12.5,1,14.8,8.4,1,14.2,2.3,1,169.7,35.1,1,100.3,41.8,1,1.0,0.7,1,1.3,1.1,1,0.8,1.1,1,0.6,0.3,1,3
609,2024-01-01 00:04:00.900,40.5,13,20,14,163,88,0.9,1.0,0.5,0.5,43.9,9.9,1,16.2,12.5,1,14.8,8.4,1,14.2,2.3,1,169.7,35.1,1,100.3,41.8,1,1.0,0.7,1,1.3,1.1,1,0.8,1.1,1,0.6,0.3,1,3


In [24]:
# normal the first pattern range data
dfnorm.iloc[conf.POINTS["MOVING"] : conf.POINTS["MOVING"] + conf.POINTS["PATTERN"]]

,DATETM,TEM,PM1,PM2_5,PM10,CO,NH3,CT1,CT2,CT3,CT4,MVAVG_TEM,MVSTD_TEM,MVSIG_TEM,MVAVG_PM1,MVSTD_PM1,MVSIG_PM1,MVAVG_PM2_5,MVSTD_PM2_5,MVSIG_PM2_5,MVAVG_PM10,MVSTD_PM10,MVSIG_PM10,MVAVG_CO,MVSTD_CO,MVSIG_CO,MVAVG_NH3,MVSTD_NH3,MVSIG_NH3,MVAVG_CT1,MVSTD_CT1,MVSIG_CT1,MVAVG_CT2,MVSTD_CT2,MVSIG_CT2,MVAVG_CT3,MVSTD_CT3,MVSIG_CT3,MVAVG_CT4,MVSTD_CT4,MVSIG_CT4,FLAG
600,2024-01-01 00:01:25.000,40.8,10,12,13,163,88,0.8,1.0,0.4,0.6,40.8,0.0,1,10.3,1.0,1,12.1,0.3,1,13.2,0.6,1,161.9,3.9,1,88.2,0.6,1,0.8,0.1,1,1.0,0.1,1,0.5,0.1,1,0.5,0.1,1,0
601,2024-01-01 00:01:25.100,40.8,10,12,13,163,88,0.8,1.1,0.5,0.6,40.8,0.0,1,10.3,1.0,1,12.1,0.3,1,13.2,0.6,1,161.9,3.9,1,88.2,0.6,1,0.8,0.1,1,1.0,0.1,1,0.5,0.1,1,0.5,0.1,1,0
602,2024-01-01 00:01:25.200,40.8,10,12,13,164,88,0.8,1.0,0.5,0.6,40.8,0.0,1,10.3,1.0,1,12.1,0.3,1,13.2,0.6,1,161.9,3.9,1,88.2,0.6,1,0.8,0.1,1,1.0,0.1,1,0.5,0.1,1,0.5,0.1,1,0
603,2024-01-01 00:01:25.300,40.8,10,12,13,164,88,0.8,1.0,0.5,0.5,40.8,0.0,1,10.3,1.0,1,12.1,0.3,1,13.2,0.6,1,161.9,3.9,1,88.2,0.6,1,0.8,0.1,1,1.0,0.1,1,0.5,0.1,1,0.5,0.1,1,0
604,2024-01-01 00:01:25.400,40.8,10,12,13,164,88,0.8,1.1,0.6,0.5,40.8,0.0,1,10.3,1.0,1,12.1,0.3,1,13.2,0.6,1,161.9,3.9,1,88.2,0.6,1,0.8,0.1,1,1.0,0.1,1,0.5,0.1,1,0.5,0.1,1,0
605,2024-01-01 00:01:25.500,40.8,10,12,13,164,87,0.8,0.9,0.5,0.5,40.8,0.0,1,10.3,1.0,1,12.1,0.3,1,13.2,0.6,1,161.9,3.9,1,88.2,0.6,1,0.8,0.1,1,1.0,0.1,1,0.5,0.1,1,0.5,0.1,1,0
606,2024-01-01 00:01:25.600,40.8,10,12,13,164,87,0.8,1.1,0.5,0.5,40.8,0.0,1,10.3,1.0,1,12.1,0.3,1,13.2,0.6,1,161.9,3.9,1,88.2,0.6,1,0.8,0.1,1,1.0,0.1,1,0.5,0.1,1,0.5,0.1,1,0
607,2024-01-01 00:01:25.700,40.8,10,12,13,164,87,0.9,1.0,0.4,0.5,40.8,0.0,1,10.3,1.0,1,12.1,0.3,1,13.2,0.6,1,161.9,3.9,1,88.2,0.6,1,0.8,0.1,1,1.0,0.1,1,0.5,0.1,1,0.5,0.1,1,0
608,2024-01-01 00:01:25.800,40.9,13,13,15,164,88,0.8,1.0,0.6,0.6,40.8,0.0,1,10.3,1.0,1,12.1,0.3,1,13.2,0.6,1,161.9,3.9,1,88.2,0.6,1,0.8,0.1,1,1.0,0.1,1,0.5,0.1,1,0.5,0.1,1,0
609,2024-01-01 00:01:25.900,40.9,13,13,15,164,88,0.9,1.1,0.5,0.6,40.8,0.0,1,10.3,1.0,1,12.1,0.3,1,13.2,0.6,1,161.9,3.9,1,88.2,0.6,1,0.8,0.1,1,1.0,0.1,1,0.5,0.1,1,0.5,0.1,1,0


### Prepare mix dataframe from normal and outlier dataframe

In [25]:
# prepare mix dataframe based on normal and outlier dataframe
dfnorm["FLAG"] = 0

dfmix = pd.concat([dfnorm, dfoutl])
dfmix = dfmix.sort_values(by=conf.COLUMN_NAMES[0])
dfmix = dfmix.reset_index(drop=True)

In [26]:
# recreate work table base one dfmix
con.execute("DROP TABLE IF EXISTS work")
con.execute("CREATE TABLE work AS SELECT * FROM dfmix WHERE 1=0")

In [27]:
# insert into in-memory work table from dfmix with calculated Moving Avg,Std and asis Flag value

coldef = ", ".join(conf.COLUMN_NAMES)

for col in conf.COLUMN_GRAPH:
    coldef += (
        ", "
        + f"AVG({col}) OVER (ORDER BY {conf.COLUMN_NAMES[0]} ROWS BETWEEN {conf.POINTS['MOVING']} PRECEDING AND CURRENT ROW) AS {conf.MVAVG}{col}"
    )
    coldef += (
        ", "
        + f"STDDEV({col}) OVER (ORDER BY {conf.COLUMN_NAMES[0]} ROWS BETWEEN {conf.POINTS['MOVING']} PRECEDING AND CURRENT ROW) AS {conf.MVSTD}{col}"
    )
    coldef += ", 0"  # mvsig_
coldef += ", FLAG"  # flag

query = f"INSERT INTO work SELECT {coldef} FROM dfmix ORDER BY {conf.COLUMN_NAMES[0]}"
# print(textwrap.fill(query, width=120))

con.execute(query)

In [28]:
# Update NULL value after window function
query = f"SELECT * FROM work ORDER BY {conf.COLUMN_NAMES[0]}"
dfmix = con.execute(query).df()
dfmix.bfill(inplace=True)

# recreate work table base one work dataframe
con.execute("DROP TABLE IF EXISTS work")
con.execute("CREATE TABLE work AS SELECT * FROM dfmix")

In [29]:
# calculate Sigma value

coldef = ""
for col in conf.COLUMN_GRAPH:
    setdef = textwrap.dedent(f"""{conf.MVSIG}{col} = CASE 
                  WHEN {col} >= ({col} - 1 * {conf.MVSTD}{col}) AND {col} <= ({col} + 1 * {conf.MVSTD}{col})  then 1
                  WHEN {col} >= ({col} - 2 * {conf.MVSTD}{col}) AND {col} <= ({col} + 2 * {conf.MVSTD}{col})  then 2
                  WHEN {col} >= ({col} - 3 * {conf.MVSTD}{col}) AND {col} <= ({col} + 3 * {conf.MVSTD}{col})  then 3
                  WHEN {col} >= ({col} - 4 * {conf.MVSTD}{col}) AND {col} <= ({col} + 4 * {conf.MVSTD}{col})  then 4
                  WHEN {col} >= ({col} - 5 * {conf.MVSTD}{col}) AND {col} <= ({col} + 5 * {conf.MVSTD}{col})  then 5
                  ELSE 6
                  END""")
    if not coldef:
        coldef = setdef
    else:
        coldef += f", {setdef}"

query = f"UPDATE work SET {coldef}"
# print(textwrap.fill(query, width=120))

con.execute(query)

In [30]:
# calculate Sigma value

coldef = ""
for col in conf.COLUMN_GRAPH:
    setdef = textwrap.dedent(f"""{conf.MVSIG}{col} = CASE 
                  WHEN {col} >= ({col} - 1 * {conf.MVSTD}{col}) AND {col} <= ({col} + 1 * {conf.MVSTD}{col})  then 1
                  WHEN {col} >= ({col} - 2 * {conf.MVSTD}{col}) AND {col} <= ({col} + 2 * {conf.MVSTD}{col})  then 2
                  WHEN {col} >= ({col} - 3 * {conf.MVSTD}{col}) AND {col} <= ({col} + 3 * {conf.MVSTD}{col})  then 3
                  WHEN {col} >= ({col} - 4 * {conf.MVSTD}{col}) AND {col} <= ({col} + 4 * {conf.MVSTD}{col})  then 4
                  WHEN {col} >= ({col} - 5 * {conf.MVSTD}{col}) AND {col} <= ({col} + 5 * {conf.MVSTD}{col})  then 5
                  ELSE 6
                  END""")
    if not coldef:
        coldef = setdef
    else:
        coldef += f", {setdef}"

query = f"UPDATE work SET {coldef}"
# print(textwrap.fill(query, width=120))

con.execute(query)

In [31]:
dfmix = con.execute(f"SELECT * FROM work ORDER BY {conf.COLUMN_NAMES[0]}").df()

# round float type value
dfmix = dfmix.round(1)

In [32]:
# check data
print(
    f"row count, mix={len(dfmix)}, normal={len(dfmix[dfmix['FLAG']==0])}, outlier={len(dfmix[dfmix['FLAG']!=0])}, outlier ratio={len(dfmix[dfmix['FLAG']!=0])/len(dfmix)}"
)
dfmix.head()

row count, mix=108000, normal=81000, outlier=27000, outlier ratio=0.25


,DATETM,TEM,PM1,PM2_5,PM10,CO,NH3,CT1,CT2,CT3,CT4,MVAVG_TEM,MVSTD_TEM,MVSIG_TEM,MVAVG_PM1,MVSTD_PM1,MVSIG_PM1,MVAVG_PM2_5,MVSTD_PM2_5,MVSIG_PM2_5,MVAVG_PM10,MVSTD_PM10,MVSIG_PM10,MVAVG_CO,MVSTD_CO,MVSIG_CO,MVAVG_NH3,MVSTD_NH3,MVSIG_NH3,MVAVG_CT1,MVSTD_CT1,MVSIG_CT1,MVAVG_CT2,MVSTD_CT2,MVSIG_CT2,MVAVG_CT3,MVSTD_CT3,MVSIG_CT3,MVAVG_CT4,MVSTD_CT4,MVSIG_CT4,FLAG
0,2024-01-01 00:00:00.000,41.0,10,12,13,162,88,0.9,1.0,0.5,0.6,41.0,0.4,1,10.0,0.0,1,12.0,0.7,1,13.0,0.7,1,162.0,0.0,1,88.0,0.0,1,0.9,0.0,1,1.0,0.0,1,0.5,0.1,1,0.6,0.1,1,1
1,2024-01-01 00:00:00.100,41.5,10,13,14,162,88,0.9,1.0,0.4,0.5,41.2,0.4,1,10.0,0.0,1,12.5,0.7,1,13.5,0.7,1,162.0,0.0,1,88.0,0.0,1,0.9,0.0,1,1.0,0.0,1,0.5,0.1,1,0.6,0.1,1,1
2,2024-01-01 00:00:00.200,42.0,12,13,14,163,89,0.9,1.0,0.4,0.6,41.5,0.5,1,10.7,1.2,1,12.7,0.6,1,13.7,0.6,1,162.3,0.6,1,88.3,0.6,1,0.9,0.0,1,1.0,0.0,1,0.4,0.1,1,0.6,0.1,1,1
3,2024-01-01 00:00:00.300,43.6,11,13,14,163,89,0.9,1.0,0.5,0.6,42.0,1.1,1,10.8,1.0,1,12.8,0.5,1,13.8,0.5,1,162.5,0.6,1,88.5,0.6,1,0.9,0.0,1,1.0,0.0,1,0.5,0.1,1,0.6,0.1,1,1
4,2024-01-01 00:00:00.400,43.9,13,13,14,161,88,0.8,1.0,0.5,0.5,42.4,1.3,1,11.2,1.3,1,12.8,0.4,1,13.8,0.4,1,162.2,0.8,1,88.4,0.5,1,0.9,0.0,1,1.0,0.0,1,0.5,0.1,1,0.6,0.1,1,1


### Save dataframe into duckdb table

In [33]:
# save noise
_start = time.time()

comm.save_dftab(dfnoise, conf.TABNAME_NOISE)

_elapsed = time.time() - _start
print(f"save db, noise elapsed time: {human.format_timespan(_elapsed)}")

save db, noise elapsed time: 0.04 seconds


In [34]:
# save normal
_start = time.time()

comm.save_dftab(dfnorm, conf.TABNAME_NORM)

_elapsed = time.time() - _start
print(f"save db, norm elapsed time: {human.format_timespan(_elapsed)}")

save db, norm elapsed time: 0.13 seconds


In [35]:
# save outlier
_start = time.time()

comm.save_dftab(dfoutl, conf.TABNAME_OUTL)

_elapsed = time.time() - _start
print(f"save db, outl elapsed time: {human.format_timespan(_elapsed)}")

save db, outl elapsed time: 0.12 seconds


In [36]:
# save mix
_start = time.time()

comm.save_dftab(dfmix, conf.TABNAME_MIX)

_elapsed = time.time() - _start
print(f"save db, mix elapsed time: {human.format_timespan(_elapsed)}")

save db, mix elapsed time: 0.18 seconds


In [37]:
# close in-memory db
con.close()

### Save dataframe into csvfile

In [38]:
# save noise
_start = time.time()

comm.save_csvfile(dfnoise[conf.COLUMN_NAMES], conf.FILENAME_NOISE, conf.DIROUT)  # only noise

_elapsed = time.time() - _start
print(f"save csvfile, noise elapsed time: {human.format_timespan(_elapsed)}")  # no files for conf.INPUT_MAXSIZE=400 MB

save_csvfile, dataframe has no rows to write, skip
save csvfile, noise elapsed time: 0 seconds


In [39]:
# save normal
_start = time.time()

comm.save_csvfile(dfnorm[conf.COLUMN_NAMES], conf.FILENAME_NORM, conf.DIROUT)

_elapsed = time.time() - _start
print(f"save csvfile, norm elapsed time: {human.format_timespan(_elapsed)}")
# 11 min. 49 sec, 274 files, 380 MB for conf.INPUT_MAXSIZE=650 MB, include DATETM

saved 1/3 csvfile=sample\dataout\ohtnorm\ohtnorm-001.csv
saved 3/3 files
save csvfile, norm elapsed time: 4.52 seconds


In [40]:
# save outlier
_start = time.time()

comm.save_csvfile(dfoutl[conf.COLUMN_NAMES + ["FLAG"]], conf.FILENAME_OUTL, conf.DIROUT)  # with FLAG column

_elapsed = time.time() - _start
print(f"save csvfile, outl elapsed time: {human.format_timespan(_elapsed)}")
# 3 min. 58 sec. 72 files 128 MB for conf.INPUT_MAXSIZE=650 MB

saved 1/1 csvfile=sample\dataout\ohtoutl\ohtoutl-001.csv
saved 1/1 files
save csvfile, outl elapsed time: 1.51 seconds


In [41]:
# save mix
_start = time.time()

comm.save_csvfile(dfmix[conf.COLUMN_NAMES + ["FLAG"]], conf.FILENAME_MIX, conf.DIROUT)  # with FLAG column

_elapsed = time.time() - _start
print(f"save csvfile, mix elapsed time: {human.format_timespan(_elapsed)}")

saved 1/3 csvfile=sample\dataout\ohtmix\ohtmix-001.csv
saved 3/3 files
save csvfile, mix elapsed time: 6.57 seconds


### Check row count & file size

In [42]:
# display row count for check
print(f"row count noise={len(dfnoise)}, norm={len(dfnorm)}, outl={len(dfoutl)}, mix={len(dfmix)}")

row count noise=0, norm=81000, outl=27000, mix=108000


In [43]:
# display file size for check

fileinfo = dict()  # count, size
for afile in [conf.FILENAME_NOISE, conf.FILENAME_NORM, conf.FILENAME_OUTL, conf.FILENAME_MIX]:
    basename = pathlib.Path(afile).stem
    adir = str(pathlib.Path(conf.DIROUT) / basename)
    files = comm.get_multifiles_indir(adir, "*.csv")
    sizes = comm.get_multifiles_size(files)

    fileinfo[basename] = [len(files), sum(sizes)]

total_count, total_size = 0, 0
for basename, count_size in fileinfo.items():
    print(f"output {basename} files={count_size[0]}, size={human.format_size(count_size[1])}")
    total_count = total_count + count_size[0]
    total_size = total_size + count_size[1]

print(f"total files={total_count}, size={human.format_size(total_size)}")

output ohtnoise files=0, size=0 bytes
output ohtnorm files=3, size=5.02 MB
output ohtoutl files=1, size=1.73 MB
output ohtmix files=3, size=6.92 MB
total files=7, size=13.67 MB


In [44]:
_elapsed = time.time() - mainstart
print(f"main elapsed time: {human.format_timespan(_elapsed)}")
# 3 min.  for conf.INPUT_MAXSIZE=400MB, when include DATETM

main elapsed time: 23.72 seconds


## eof